# MATH2970 Computer Assignment

**Author:** 510575452

In [4]:
# Importing Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.optimize import minimize

## Exploring Data

In [7]:
stocks = pd.read_csv('data/project_data.csv', parse_dates=['Date'])
stocks.head()

,Date,BHP,CBA,CSL,NAB,WBC,ANZ,MQG,WES,GMG,FMG
0,2022-07-01,33.62,81.91,260.65,24.64,17.21,19.16,151.74,38.63,17.75,13.25
1,2022-07-04,33.49,82.59,266.04,24.85,17.43,19.40,154.43,39.60,18.25,13.31
2,2022-07-05,33.57,82.39,270.05,24.77,17.43,19.33,155.85,39.82,18.08,13.50
3,2022-07-06,31.69,83.09,277.01,25.22,17.59,19.51,156.75,40.47,18.77,12.85
4,2022-07-07,32.68,84.06,277.85,25.22,17.72,19.86,157.67,41.04,18.69,13.41


## Question 1

### Part A

In [25]:
# Calculate the mean return of each stock
# for stock in stocks.columns[1:]:
#     stock_return = (stocks[stock]/stocks[stock].shift(1) - 1)
#     stock_return = stock_return[1:]
#     stock_return_mean = np.mean(stock_return)
#     print("Mean return for", stock, "=", stock_return_mean)

# Calculate the mean return of each stock
for stock in stocks.columns[1:]:
    stock_return = stocks.loc[:, stock].pct_change()
    stock_return_mean = stock_return.mean()
    print("Mean return for", stock, "=", stock_return_mean)

Mean return for BHP = 0.0005290007744741171
Mean return for CBA = 0.0008938058392775244
Mean return for CSL = 0.0002944352613775068
Mean return for NAB = 0.0008256029027737885
Mean return for WBC = 0.0009736330340698034
Mean return for ANZ = 0.0008275672040342604
Mean return for MQG = 0.000686245046964906
Mean return for WES = 0.0010766100857014815
Mean return for GMG = 0.0014718806874362148
Mean return for FMG = 0.0010636958339187278


In [27]:
# Calculate the covariance matrix
covariant_matrix = stocks.iloc[:, 1:].pct_change()
covariant_matrix.cov()

,BHP,CBA,CSL,NAB,WBC,ANZ,MQG,WES,GMG,FMG
BHP,0.000224,0.000038,0.000004,0.000035,0.000036,0.000034,0.000074,0.000046,0.000052,0.000230
CBA,0.000038,0.000106,0.000041,0.000091,0.000090,0.000080,0.000077,0.000063,0.000075,0.000040
CSL,0.000004,0.000041,0.000123,0.000031,0.000029,0.000029,0.000045,0.000051,0.000064,0.000005
NAB,0.000035,0.000091,0.000031,0.000120,0.000099,0.000089,0.000081,0.000059,0.000066,0.000040
WBC,0.000036,0.000090,0.000029,0.000099,0.000126,0.000095,0.000079,0.000061,0.000066,0.000044
ANZ,0.000034,0.000080,0.000029,0.000089,0.000095,0.000115,0.000078,0.000058,0.000068,0.000043
MQG,0.000074,0.000077,0.000045,0.000081,0.000079,0.000078,0.000185,0.000086,0.000113,0.000096
WES,0.000046,0.000063,0.000051,0.000059,0.000061,0.000058,0.000086,0.000135,0.000113,0.000048
GMG,0.000052,0.000075,0.000064,0.000066,0.000066,0.000068,0.000113,0.000113,0.000279,0.000075
FMG,0.000230,0.000040,0.000005,0.000040,0.000044,0.000043,0.000096,0.000048,0.000075,0.000430


## Question 2